In [1]:
import os
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets import mnist
import numpy as np

In [2]:
save_dir = "mnist"

In [3]:
data_sets = mnist.read_data_sets(save_dir,
                                 dtype=tf.uint8,
                                 reshape=False,
                                 validation_size=1000)

Extracting mnist\train-images-idx3-ubyte.gz
Extracting mnist\train-labels-idx1-ubyte.gz
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz


In [4]:
data_splits = ["train","test","validation"]
for d in range(len(data_splits)):
    print("saving " + data_splits[d])
    data_set = data_sets[d]

    filename = os.path.join(save_dir, data_splits[d] + '.tfrecords')
    writer = tf.python_io.TFRecordWriter(filename)
    
    for index in range(data_set.images.shape[0]):
        image = data_set.images[index].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
        'height': tf.train.Feature(int64_list=
                                  tf.train.Int64List(value=
                                  [data_set.images.shape[1]])),
        'width': tf.train.Feature(int64_list=
                                  tf.train.Int64List(value =
                                  [data_set.images.shape[2]])),
        'depth': tf.train.Feature(int64_list=
                                  tf.train.Int64List(value =
                                  [data_set.images.shape[3]])),
        'label': tf.train.Feature(int64_list=
                                  tf.train.Int64List(value =
                                  [int(data_set.labels[index])])),
        'image_raw': tf.train.Feature(bytes_list=
                                   tf.train.BytesList(value =
                                                     [image]))}))
    writer.write(example.SerializeToString())
writer.close()

saving train
saving test
saving validation


In [5]:
#tf.train.string_input_producer() creates a queueRunner. It outputs filename strings to a
#queue for our input  pipeline
#num_epochs - produces each file name strings num_epoch times
filename = os.path.join(save_dir ,"train.tfrecords")
filename_queue = tf.train.string_input_producer(
    [filename], num_epochs=10)

In [6]:
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
features = tf.parse_single_example(
    serialized_example,
    features={
        'image_raw': tf.FixedLenFeature([], tf.string),
        'label': tf.FixedLenFeature([], tf.int64),
    })

In [7]:
image = tf.decode_raw(features['image_raw'], tf.uint8)

In [8]:
image.set_shape([784]) 

In [9]:
image = tf.cast(image, tf.float32) * (1. / 255)

In [10]:
label = tf.cast(features['label'], tf.int32)

In [27]:
images_batch, labels_batch = tf.train.shuffle_batch(
    [image, label], batch_size=128,
    capacity=2000,
    min_after_dequeue=1000)

In [28]:
W = tf.get_variable("W",[28*28,10])

ValueError: Variable W already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "C:\Users\ZekeLabs\Anaconda3-N\lib\site-packages\tensorflow\python\framework\ops.py", line 1204, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\ZekeLabs\Anaconda3-N\lib\site-packages\tensorflow\python\framework\ops.py", line 2630, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "C:\Users\ZekeLabs\Anaconda3-N\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)


In [29]:
y_pred = tf.matmul(images_batch,W)

In [30]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_pred,labels=labels_batch)

In [31]:
loss_mean = tf.reduce_mean(loss)

In [32]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [33]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
sess.run(tf.local_variables_initializer())

In [34]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess,coord=coord)

In [35]:
threads

[<Thread(Thread-12, stopped daemon 12136)>,
 <Thread(Thread-13, started daemon 12348)>,
 <Thread(Thread-14, stopped daemon 12392)>,
 <Thread(Thread-15, started daemon 21800)>,
 <Thread(Thread-16, stopped daemon 9060)>,
 <Thread(Thread-17, started daemon 21156)>,
 <Thread(Thread-18, stopped daemon 19012)>,
 <Thread(Thread-19, started daemon 19932)>]

In [36]:
try:
    step = 0
    while not coord.should_stop():
        step += 1
        print ('here')
        sess.run([train_op])
        if step%10:
            loss_mean_val = sess.run([loss_mean])
            print (step)
            print (loss_mean_val)
except:
    print ('Done')
            

here
Done


In [26]:
sess.run(train_op)

OutOfRangeError: RandomShuffleQueue '_1_shuffle_batch_1/random_shuffle_queue' is closed and has insufficient elements (requested 128, current size 5)
	 [[Node: shuffle_batch_1 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch_1/random_shuffle_queue, shuffle_batch_1/n)]]

Caused by op 'shuffle_batch_1', defined at:
  File "C:\Users\ZekeLabs\Anaconda3-N\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\ZekeLabs\Anaconda3-N\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\ZekeLabs\Anaconda3-N\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\ZekeLabs\Anaconda3-N\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\ZekeLabs\Anaconda3-N\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoo